In [25]:
import pandas as pd
import pickle
import json
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from textblob import TextBlob
from translate import Translator
import os
import dask.dataframe as dd

In [26]:
with open('dialogos.pickle', "rb") as f:
    dialogos_completos = pickle.load(f)

In [27]:
#szigriszt_pazos cuanto más cerca de 100, más sencillo es

In [28]:
df = pd.DataFrame(dialogos_completos)
df['hablante'] = df['hablante'].apply(json.dumps)
df = pd.concat([df, pd.json_normalize(df['hablante'].apply(json.loads))], axis=1).drop(columns=['hablante'])
df

,discurso,fecha_proceso,szigriszt_pazos,flesch_reading_ease,nombre,grupo parlamentario
0,"Señor Sánchez, usted bloquea hasta la verdad ...",2020-02-19,-79.60,-79.01,"Casado Blanco, Pablo",Grupo Parlamentario Popular en el Congreso
1,"Muchas gracias, señora presidenta Claro que n...",2020-02-19,32.24,35.84,"Sánchez Pérez-Castejón, Pedro",Grupo Parlamentario Socialista
2,"Señor Sánchez, sus recetas económicas son tan...",2020-02-19,-60.71,-59.39,"Casado Blanco, Pablo",Grupo Parlamentario Popular en el Congreso
3,"Muchas gracias, señora presidenta Señor Casad...",2020-02-19,-78.36,-77.63,"Sánchez Pérez-Castejón, Pedro",Grupo Parlamentario Socialista
4,"Gracias, señora presidenta Señor Sánchez, el ...",2020-02-19,-0.88,2.24,"Arrimadas García, Inés",Grupo Parlamentario Ciudadanos
...,...,...,...,...,...,...
19157,"Termino Como le dije, doña Isabel, su discurs...",2021-05-11,74.03,78.20,"Mestre Barea, Manuel",Grupo Parlamentario VOX
19158,"Muchas gracias, señora presidenta Señorías, m...",2021-05-11,-804.20,-817.55,"Gutiérrez Díaz de Otazu, Fernando Adolfo",Grupo Parlamentario Popular en el Congreso
19159,con las mayores garantías de éxito y de segur...,2021-05-11,50.67,55.52,"Gutiérrez Díaz de Otazu, Fernando Adolfo",Grupo Parlamentario Popular en el Congreso
19160,"Gracias, señora presidenta Señorías, antes de...",2021-05-11,-65.56,-64.45,"Cerdán León, Santos",Grupo Parlamentario Socialista


In [30]:
resultados = df.groupby(['grupo parlamentario']).agg({'szigriszt_pazos': ['max', 'min']}).reset_index()

# renombramos las columnas del resultado
resultados.columns = ['grupo parlamentario', 'max_szigriszt_pazos', 'min_szigriszt_pazos']

# hacemos un merge de los resultados con el dataframe original para conservar todas las columnas
resultados = pd.merge(df, resultados, on='grupo parlamentario')


In [31]:
medias = df.groupby(['grupo parlamentario', 'fecha_proceso'])['szigriszt_pazos'].mean().reset_index()


In [32]:
lista_grupos = ['Grupo Parlamentario Ciudadanos', 
                'Grupo Parlamentario Confederal de Unidas Podemos-En Comú Podem-Galicia en Común',
                'Grupo Parlamentario Popular en el Congreso',
                'Grupo Parlamentario Socialista',
                'Grupo Parlamentario VOX'
               ]
df_medias = medias[medias['grupo parlamentario'].isin(lista_grupos)]

In [33]:
import os
import re
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource
from bokeh.io import output_notebook

output_notebook()  # para mostrar la gráfica en el notebook

    
colores_grupos = {
    'GCs':'#DF6524',
    'GCUP_EC_GC':'#693279',
    'GP': '#184A90',
    'GS': '#DC0612',
    'GVOX': '#63BE21'
}
    

# Crear un objeto ColumnDataSource con los datos
source_GCs = ColumnDataSource(data=dict(
    fecha=df_medias[df_medias['grupo parlamentario'] == 'Grupo Parlamentario Ciudadanos']['fecha_proceso'],
    szigriszt_pazos=df_medias[df_medias['grupo parlamentario'] == 'Grupo Parlamentario Ciudadanos']['szigriszt_pazos']
))

source_GCUP_EC_GC = ColumnDataSource(data=dict(
    fecha=df_medias[df_medias['grupo parlamentario'] == 
                    'Grupo Parlamentario Confederal de Unidas Podemos-En Comú Podem-Galicia en Común']['fecha_proceso'],
    szigriszt_pazos=df_medias[df_medias['grupo parlamentario'] == 
                        'Grupo Parlamentario Confederal de Unidas Podemos-En Comú Podem-Galicia en Común']['szigriszt_pazos']
))

source_GP = ColumnDataSource(data=dict(
    fecha=df_medias[df_medias['grupo parlamentario'] == 'Grupo Parlamentario Popular en el Congreso']['fecha_proceso'],
    szigriszt_pazos=df_medias[df_medias['grupo parlamentario'] == 'Grupo Parlamentario Popular en el Congreso']['szigriszt_pazos']
))

source_GS = ColumnDataSource(data=dict(
    fecha=df_medias[df_medias['grupo parlamentario'] == 'Grupo Parlamentario Socialista']['fecha_proceso'],
    szigriszt_pazos=df_medias[df_medias['grupo parlamentario'] == 'Grupo Parlamentario Socialista']['szigriszt_pazos']
))

source_GVOX = ColumnDataSource(data=dict(
    fecha=df_medias[df_medias['grupo parlamentario'] == 'Grupo Parlamentario VOX']['fecha_proceso'],
    szigriszt_pazos=df_medias[df_medias['grupo parlamentario'] == 'Grupo Parlamentario VOX']['szigriszt_pazos']
))

# Crear una figura de Bokeh
p = figure(title="Complejidad de discurso por fecha", x_axis_label="Fecha", y_axis_label="szigriszt_pazos",
           x_range=df_medias["fecha_proceso"].drop_duplicates().sort_values(ascending=True), plot_width=2400, plot_height=600)
p.xaxis.major_label_orientation = 45

# Añadir la línea a la figura
p.line(x="fecha", y="szigriszt_pazos", source=source_GCs, line_color=colores_grupos['GCs'], line_width=2, legend_label="GCs")
p.line(x="fecha", y="szigriszt_pazos", source=source_GCUP_EC_GC, line_color=colores_grupos['GCUP_EC_GC'], line_width=2, legend_label="GCUP_EC_GC")
p.line(x="fecha", y="szigriszt_pazos", source=source_GP, line_color=colores_grupos['GP'], line_width=2, legend_label="GP")
p.line(x="fecha", y="szigriszt_pazos", source=source_GS, line_color=colores_grupos['GS'], line_width=2, legend_label="GS")
p.line(x="fecha", y="szigriszt_pazos", source=source_GVOX, line_color=colores_grupos['GVOX'], line_width=2, legend_label="GVOX")

# Añadir la leyenda a la figura
p.legend.location = "bottom_right"
p.legend.click_policy="hide"

# Mostrar la figura
show(p)

Loading BokehJS ...

In [9]:
# 20.02.2020 DELEGACIÓN EN LA MESA DE LA COMISIÓN DE LA COMPETENCIA DE ESTA DE ADOPCIÓN DE LOS ACUERDOS A QUE SE REFIERE EL ARTÍCULO 44 DEL REGLAMENTO, CONCORDANTE CON LA RESOLUCIÓN DE LA PRESIDENCIA DE LA CÁMARA DE 2 DE NOVIEMBRE DE 1983.
# 18.03.2020 Comparecencia del presidente del Gobierno en el Congreso de los Diputados por el Estado de Alarma en la crisis del coronavirus. .
# 25.03.2020 E SE DEROGA EL DESPIDO OBJETIVO POR FALTAS DE ASISTENCIA AL TRABAJO ESTABLECIDO EN EL ARTÍCULO 52. D) DEL TEXTO REFUNDIDO DE LA LEY DEL ESTATUTO DE LOS TRABAJADORES,
# 09.04.2020 Prórroga del estado de alarma
# 16.05.2020 Prórroga del estado de alarma
# 14.07.2020 Proyecto de ley de cambio climático y transición energética.
# 20.10.2020 Moción de censura, candidato SAntiago Abascal
# 28.01.2021 Proposición de Ley Orgánica por la que se modifica la Ley Orgánica 6/1985, de 1 de julio, del Poder Judicial, para el establecimiento del régimen jurídico aplicable al Consejo General del Poder Judicial en funciones.
#

In [34]:
import textstat
textstat.set_lang('es')
textstat.szigriszt_pazos('hola, ¿como te llamas?')

93.82

In [35]:
texto = 'La polimerización se detiene destruyendo los radicales libres presentes por medio de la adición de un estabilizante de acción rápida. Luego de remover el monómero que no ha reaccionado, el polímero es asilado desestabilizando el sistema coloidal, separando la fase acuosa y secando el polímero.'
textstat.szigriszt_pazos(texto)

38.97

Un valor negativo en la media del valor de Szigriszt-Pazos de un grupo parlamentario indica que los discursos de los diputados de ese grupo parlamentario son en promedio difíciles de leer y comprenden para una audiencia general. Esto podría deberse a varios factores, como el uso de un lenguaje técnico o jerga, la complejidad de las oraciones o la falta de claridad en la exposición.

En un informe, podrías destacar que el valor negativo en la media de Szigriszt-Pazos sugiere que los discursos de los diputados de ciertos grupos parlamentarios pueden no estar comunicando efectivamente sus ideas a una audiencia amplia, lo que podría tener un impacto negativo en su capacidad para persuadir a otros y para obtener el apoyo público.

También podrías mencionar que la media del valor de Szigriszt-Pazos por sí sola no proporciona información completa sobre la calidad de los discursos de los diputados, y que es importante considerar otros factores, como el contenido y la relevancia de las ideas expresadas en los discursos, para hacer una evaluación completa de su efectividad.


In [36]:
def devuelve_polaridad(row):
    #como no me fio de los modelos calculamos la polaridad en 2 de ellos y calculamos la media
    translator= Translator(from_lang='es', to_lang='en')
    sentiment = SentimentIntensityAnalyzer()
    try:
        #la polaridad va entre -1 y +1
        translation = translator.translate(row['lemma'].strip().lower()).strip().lower()
        return (TextBlob(translation).sentiment.polarity + sentiment.polarity_scores(translation)['compound']) / 2
    except:
        return 0.0


In [37]:
palabras_lemm = 'palabras.pickle'
with open(palabras_lemm, "rb") as f:
    df_lemmas = pickle.load(f)

In [38]:
%%time
if os.path.exists('nuevas_polaridades.pickle'):
     with open('nuevas_polaridades.pickle', "rb") as f:
            nuevas_polaridades = pickle.load(f)
else:
    nuevas_polaridades = df_lemmas['palabra'].drop_duplicates().reset_index().drop('index', axis=1)
    print(len(nuevas_polaridades))
    #recalculamos polaridades con el nuevo método
    #nuevas_polaridades['polaridad'] = nuevas_polaridades.apply(devuelve_polaridad, axis=1)
    # crear un DataFrame Dask a partir del DataFrame de Pandas
    meta = pd.Series([], dtype='float64')
    df_dask = dd.from_pandas(nuevas_polaridades, npartitions=10)

    # aplicar la función devuelve_polaridad en paralelo a cada partición del DataFrame
    df_dask['polaridad'] = df_dask.map_partitions(lambda df: df.apply(devuelve_polaridad, axis=1), meta=meta)

    # convertir el DataFrame Dask de vuelta a un DataFrame de Pandas
    nuevas_polaridades = df_dask.compute()
    
    with open('nuevas_polaridades.pickle', "wb") as f:
        pickle.dump(nuevas_polaridades, f)
nuevas_polaridades

CPU times: total: 31.2 ms
Wall time: 21 ms


,palabra,polaridad
0,Señor,0.00000
1,Sánchez,0.00000
2,bloquea,-0.24630
3,ayer,0.00000
4,tuvimos,0.00000
...,...,...
98446,antipolígono,0.28315
98447,Congozantes,0.28315
98448,Roncal,0.28315
98449,Cédula,0.28315


In [39]:
colores_grupos = {
    'GCs':'#DF6524',
    'GCUP_EC_GC':'#693279',
    'GP': '#184A90',
    'GS': '#DC0612',
    'GVOX': '#63BE21'
}

In [41]:
import os
import re
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource
from bokeh.io import output_notebook

output_notebook()  # para mostrar la gráfica en el notebook

folder_path = 'resultados_analisis'  # Reemplaza por la ruta a la carpeta que deseas leer
#pattern = r'^COM_DISCURSO_(\d{4})_(\d{2})_(\d{2})_GS\.csv$'  # Patrón para filtrar los archivos y capturar los grupos de fecha
pattern = r'^COM_DISCURSO_(\d{4})_(\d{2})_(\d{2})_(GCs|GCUP_EC_GC|GP|GS|GVOX)\.csv$'

# Obtiene los nombres de todos los archivos en la carpeta
files = os.listdir(folder_path)

# Filtra los archivos que cumplan con el patrón
filtered_files = [f for f in files if re.match(pattern, f)]

df_salida = pd.DataFrame()
# Itera sobre los archivos filtrados
for file_name in filtered_files:
    try:
        file_path = os.path.join(folder_path, file_name)
        polaridades = pd.read_csv(file_path)
        match = re.match(pattern, file_name)
        year = match.group(1)
        month = match.group(2)
        day = match.group(3)
        grupo = match.group(4)
        date_str = f"{year}-{month}-{day}"
        polaridades['fecha'] = date_str
        polaridades['grupo'] = grupo
        #recalculamos polaridades con el nuevo método
        #polaridades= polaridades.drop('polaridad', axis=1)
        #polaridades = pd.merge(polaridades, nuevas_polaridades, on='palabra')
        media_polaridad = polaridades.groupby(['grupo', 'fecha'])['polaridad'].mean()
        media_polaridad_df = media_polaridad.reset_index()
        df_salida = pd.concat([df_salida, media_polaridad_df], axis=0)
    except pd.errors.EmptyDataError:
        continue
       

# Crear un objeto ColumnDataSource con los datos
source_GCs = ColumnDataSource(data=dict(
    fecha=df_salida[df_salida['grupo'] == 'GCs']['fecha'],
    polaridad=df_salida[df_salida['grupo'] == 'GCs']['polaridad']
))

source_GCUP_EC_GC = ColumnDataSource(data=dict(
    fecha=df_salida[df_salida['grupo'] == 'GCUP_EC_GC']['fecha'],
    polaridad=df_salida[df_salida['grupo'] == 'GCUP_EC_GC']['polaridad']
))

source_GP = ColumnDataSource(data=dict(
    fecha=df_salida[df_salida['grupo'] == 'GP']['fecha'],
    polaridad=df_salida[df_salida['grupo'] == 'GP']['polaridad']
))

source_GS = ColumnDataSource(data=dict(
    fecha=df_salida[df_salida['grupo'] == 'GS']['fecha'],
    polaridad=df_salida[df_salida['grupo'] == 'GS']['polaridad']
))

source_GVOX = ColumnDataSource(data=dict(
    fecha=df_salida[df_salida['grupo'] == 'GVOX']['fecha'],
    polaridad=df_salida[df_salida['grupo'] == 'GVOX']['polaridad']
))

# Crear una figura de Bokeh
p = figure(title="Polaridad por fecha", x_axis_label="Fecha", y_axis_label="Polaridad",
           x_range=df_salida["fecha"].drop_duplicates().sort_values(ascending=True), plot_width=2400, plot_height=600)
p.xaxis.major_label_orientation = 45

# Añadir la línea a la figura
p.line(x="fecha", y="polaridad", source=source_GCs, line_color=colores_grupos['GCs'], line_width=2, legend_label="GCs")
p.line(x="fecha", y="polaridad", source=source_GCUP_EC_GC, line_color=colores_grupos['GCUP_EC_GC'], line_width=2, legend_label="GCUP_EC_GC")
p.line(x="fecha", y="polaridad", source=source_GP, line_color=colores_grupos['GP'], line_width=2, legend_label="GP")
p.line(x="fecha", y="polaridad", source=source_GS, line_color=colores_grupos['GS'], line_width=2, legend_label="GS")
p.line(x="fecha", y="polaridad", source=source_GVOX, line_color=colores_grupos['GVOX'], line_width=2, legend_label="GVOX")

# Añadir la leyenda a la figura
p.legend.location = "bottom_right"
p.legend.click_policy="hide"

# Mostrar la figura
show(p)



Loading BokehJS ...

In [17]:
#polaridades= polaridades.drop('polaridad', axis=1)
        #polaridades = pd.merge(polaridades, nuevas_polaridades, on='palabra')
polaridades
nuevas_polaridades

,palabra,polaridad
0,Señor,0.00000
1,Sánchez,0.00000
2,bloquea,-0.24630
3,ayer,0.00000
4,tuvimos,0.00000
...,...,...
98446,antipolígono,0.28315
98447,Congozantes,0.28315
98448,Roncal,0.28315
98449,Cédula,0.28315


In [18]:
archivo_pr = 'PR_DISCURSO_GLOBAL'
df_page_rank = pd.read_csv(f'resultados_analisis\{archivo_pr}.csv').rename(columns={'total': 'PageRank'})
df_page_rank = df_page_rank[df_page_rank['grupo_parlamentario'].isin(lista_grupos)]
df_page_rank = df_page_rank.groupby(['palabra'])['PageRank'].max().reset_index()
df_page_rank.nlargest(10, 'PageRank')

,palabra,PageRank
38,hablar,75.570824
63,popular,67.110806
60,poder,60.463847
3,caso,60.187818
7,comunidad,57.078731
94,ver,56.192216
27,euros,55.313338
61,político,55.046711
78,seguir,54.779007
2,autónomo,53.667122


In [19]:
for key in colores_grupos.keys():
    archivo_pr = f'PR_DISCURSO_{key}'
    df_page_rank = pd.read_csv(f'resultados_analisis\{archivo_pr}.csv').rename(columns={'total': 'PageRank'})
    df_page_rank = df_page_rank[df_page_rank['grupo_parlamentario'].isin(lista_grupos)]
    df_page_rank = df_page_rank.groupby(['palabra'])['PageRank'].max().reset_index()
    print(key)
    print(df_page_rank.nlargest(10, 'PageRank')[['palabra', 'PageRank']])

GCs
       palabra   PageRank
6   ciudadanos  63.799610
36      hablar  52.144559
23    enmienda  49.532469
10       creer  45.918737
95         ver  45.032109
71    problema  44.783434
64       poder  42.828670
84   situación  39.627945
60       pedir  39.344560
9         cosa  39.319098
GCUP_EC_GC
       palabra   PageRank
64     popular  66.232897
40      hablar  58.459820
87  trabajador  50.775072
52       mujer  50.026091
89     trabajo  47.793713
23     empresa  43.857753
61       poder  43.212027
83   situación  38.350185
8       crisis  37.123777
18     derecha  37.067381
GP
       palabra   PageRank
64     popular  80.448450
90     sánchez  70.265461
49    ministro  57.944295
87  socialista  57.289052
48    ministra  55.955550
68  presidente  50.485118
39      hablar  48.985660
58       pedir  47.961365
28       euros  46.792535
3     autónomo  44.752147
GS
        palabra   PageRank
62      popular  71.774431
29        euros  64.593502
38       hablar  61.412251
7     comunid

In [20]:
#seleccinamos 10 fechas aleatorias excluyendo el debate de investidura
# Convertir las fechas a objetos datetime
df_fechas = df_salida["fecha"].drop_duplicates().sort_values(ascending=True).reset_index().iloc[2:]

# Calcular el paso a saltar entre las fechas
paso = int(len(df_fechas) / 10)

# Obtener las fechas igualmente espaciadas
fechas = df_fechas.iloc[::paso]['fecha']

print(fechas.to_list())

['2020-01-07', '2020-06-11', '2020-10-13', '2021-02-02', '2021-04-22', '2021-09-21', '2021-12-15', '2022-03-22', '2022-06-21', '2022-10-13']


In [21]:
df_PR_salida = pd.DataFrame()
for fec in fechas.to_list():
    #print(fec)
    df_PR_fecha = pd.DataFrame()
    f_form = fec.replace('-', '_')
    for key in colores_grupos.keys():
        archivo_pr = f'PR_DISCURSO_{f_form}_{key}'
        df_page_rank = pd.read_csv(f'resultados_analisis\{archivo_pr}.csv').rename(columns={'total': 'PageRank'})
        df_page_rank = df_page_rank[df_page_rank['grupo_parlamentario'].isin(lista_grupos)]
        df_page_rank = df_page_rank.groupby(['palabra'])['PageRank'].max().reset_index()
        archivo_com = f'COM_DISCURSO_{f_form}_{key}'
        df_comm = pd.read_csv(f'resultados_analisis\{archivo_com}.csv')
        #recalculamos polaridades con el nuevo método
        df_comm= df_comm.drop('polaridad', axis=1)
        df_comm = pd.merge(df_comm, nuevas_polaridades, on='palabra')
        output = df_page_rank.nlargest(10, 'PageRank')[['palabra', 'PageRank']].reset_index() 
        output = pd.merge(output, df_comm, on="palabra") 
        output = pd.merge(output, df_comm.groupby(['comunidad'])['polaridad'].mean().reset_index()\
                          .rename(columns={'polaridad': 'polaridad_media_' + key}), on='comunidad')
        output = output.rename(columns={'palabra': 'palabra_' + key, 
                         'PageRank': 'PageRank_' + key,
                         'polaridad': 'polaridad_' + key,
                         'comunidad': 'comunidad_' + key,
                        }).drop('index', axis=1)
        output = output[['palabra_' + key, 'PageRank_' + key, 'polaridad_' + key, 'polaridad_media_' + key, 'comunidad_' + key]]
   #     print(key)
   #     print(output)
        df_PR_fecha = pd.concat([df_PR_fecha, output], axis=1)
    df_PR_fecha['fecha'] = fec
    df_PR_salida = pd.concat([df_PR_salida, df_PR_fecha], axis=0)

In [22]:
df_comm

,palabra,comunidad,polaridad
0,acumular,100,0.28315
1,agua,100,0.28315
2,apertura,100,0.17000
3,arma,100,0.28315
4,arruinar,100,0.28315
...,...,...,...
492,sufrir,10681,0.28315
493,suicida,10681,0.28315
494,tachar,10681,0.28315
495,votación,10681,0.28315


In [23]:
comunidades = 'resultados_analisis\COM_DISCURSO_2020_01_07_GVOX.csv'
pd.read_csv(comunidades)


,palabra,polaridad,comunidad
0,autónomo,0.0,2
1,comunidad,0.0,4
2,euros,0.0,8
3,económico,0.0,9
4,acaso,0.3,129
...,...,...,...
269,servir,0.0,3588
270,sexualmente,0.3,3588
271,teruel,0.3,3588
272,viola,0.0,3588


In [24]:
#intentamo slocalizart un discurso verdaderamente negativo
df_PR_salida = pd.DataFrame()
for fec in df_fechas['fecha'].to_list():
    #print(fec)
    df_PR_fecha = pd.DataFrame()
    f_form = fec.replace('-', '_')
    for key in colores_grupos.keys():
        try:
            archivo_pr = f'PR_DISCURSO_{f_form}_{key}'
            df_page_rank = pd.read_csv(f'resultados_analisis\{archivo_pr}.csv').rename(columns={'total': 'PageRank'})
            df_page_rank = df_page_rank[df_page_rank['grupo_parlamentario'].isin(lista_grupos)]
            df_page_rank = df_page_rank.groupby(['palabra'])['PageRank'].max().reset_index()
            archivo_com = f'COM_DISCURSO_{f_form}_{key}'
            df_comm = pd.read_csv(f'resultados_analisis\{archivo_com}.csv')
            #recalculamos polaridades con el nuevo método
            df_comm= df_comm.drop('polaridad', axis=1)
            df_comm = pd.merge(df_comm, nuevas_polaridades, on='palabra')
            output = df_page_rank.nlargest(10, 'PageRank')[['palabra', 'PageRank']].reset_index() 
            output = pd.merge(output, df_comm, on="palabra") 
            output = pd.merge(output, df_comm.loc[df_comm['polaridad'] != 0.0].groupby('comunidad')['polaridad'].mean().\
                              reset_index()\
                              .rename(columns={'polaridad': 'polaridad_media_' + key}), on='comunidad')
            output = output.rename(columns={'palabra': 'palabra_' + key, 
                             'PageRank': 'PageRank_' + key,
                             'polaridad': 'polaridad_' + key,
                             'comunidad': 'comunidad_' + key,
                            }).drop('index', axis=1)
            output = output[['palabra_' + key, 'PageRank_' + key, 'polaridad_' + key, 'polaridad_media_' + key, 'comunidad_' + key]]
            if len(output[(output['polaridad_media_' + key] <= 0) & (output['polaridad_' + key] < 0)]) > 0:
                print('key: ' + key)
                print('f_form: ' + f_form)

            df_PR_fecha = pd.concat([df_PR_fecha, output], axis=1)
        except pd.errors.EmptyDataError:
            continue
    df_PR_fecha['fecha'] = fec
    df_PR_salida = pd.concat([df_PR_salida, df_PR_fecha], axis=0)
print("FIN")

KeyboardInterrupt: 

In [ ]:
df_comm[df_comm['comunidad'] == 10681].sort_values(by=['comunidad', 'polaridad'], ascending=[True, False])

In [ ]:
df_pol = pd.read_csv('resultados_analisis\COM_DISCURSO_GLOBAL.csv')
df_pol[df_pol['palabra'] == 'recurrir']

In [ ]:
polaridades

In [43]:
hablante = 'Guitarte Gimeno, Tomás'
print(f"CALL gds.graph.project.cypher( \
        'DISCURSO_TERUEL', \
        'MATCH (n:Palabra) RETURN id(n) AS id, labels(n) as labels', \
        'MATCH (p1:Palabra)-[d:DISCURSO]->(p2:Palabra) \
        WHERE d.hablante = \"{hablante}\" \
        RETURN id(p1) AS source, id(p2) AS target, type(d) AS type') \
        YIELD graphName AS graph, nodeQuery, nodeCount AS nodes, relationshipQuery, \
        relationshipCount AS rels")

CALL gds.graph.project.cypher(         'DISCURSO_TERUEL',         'MATCH (n:Palabra) RETURN id(n) AS id, labels(n) as labels',         'MATCH (p1:Palabra)-[d:DISCURSO]->(p2:Palabra)         WHERE d.hablante = "Guitarte Gimeno, Tomás"         RETURN id(p1) AS source, id(p2) AS target, type(d) AS type')         YIELD graphName AS graph, nodeQuery, nodeCount AS nodes, relationshipQuery,         relationshipCount AS rels
